<a href="https://colab.research.google.com/github/raja-jamwal/blog-agentic-architectures/blob/main/Part_2_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

## Orchestrating Intelligence: Building a Multi-Agent Supervisor

Welcome to Part 2 of the LangGraph implementation guide.

In Part 1, we built a single "Super-Node" agent.
In Part 2, we are scaling up. We will build a **Multi-Agent System**.

### The Architecture: Hub & Spoke
Instead of one agent trying to do everything, we will create:
1.  **The Supervisor (Hub):** A "Project Manager" LLM that plans and delegates.
2.  **The Workers (Spokes):** Specialized agents (Researcher, Coder) that execute tasks.

### The Stack
*   **LangGraph:** For the cyclic graph topology.
*   **LangChain:** For agent definitions.
*   **OpenAI:** GPT-4o (The Supervisor needs a smart model).

### The Goal
Handle a complex request: *"Research the current stock price of Apple and plot a chart."*
*   The **Researcher** will find the data.
*   The **Coder** will generate the plot code.
*   The **Supervisor** will manage the handoffs.

In [3]:
# @title 1. Install Dependencies
!pip install -qU langgraph langchain langchain-openai langchain-core

import os
from getpass import getpass

# @title 2. Setup API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI

# We use GPT-4o because routing requires high reasoning capabilities
llm = ChatOpenAI(model="gpt-4o", temperature=0)

print("✅ Environment Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 23.3 MB/s eta 0:00:00
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Step 1: Defining the Team State

Our state needs to track two things:
1.  `messages`: The global conversation history (so everyone sees what has been done).
2.  `next`: A string indicating **who acts next** (e.g., "Researcher", "Coder", or "FINISH").

In [1]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    # The shared memory of the team
    messages: Annotated[list, add_messages]
    # The supervisor's decision on who acts next
    next: str

print("✅ Team State Defined.")


✅ Team State Defined.


## Step 2: Creating the Workers

We need two specialized agents.
1.  **Researcher:** Has a `web_search` tool.
2.  **Coder:** Has a `python_repl` tool.

To make this easy, we'll create a helper function `create_agent` that wraps a standard LangChain agent into a graph node.

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
import random

# --- 1. Define Mock Tools ---
# In a real app, these would be real APIs (Tavily, PythonExec, etc.)

@tool
def web_search(query: str) -> list[float]:
    """Search the web for information."""
    print(f"    🔎 [Tool] Searching for: {query}")

    # Generate 10 random stock prices between 100 and 200
    prices = [round(random.uniform(100, 200), 2) for _ in range(10)]
    return prices

@tool
def python_repl(code: str):
    """Executes python code to generate charts."""
    print(f"    💻 [Tool] Executing Python: {code}")
    return "Chart generated successfully at /tmp/chart.png"

# --- 2. Helper Function to Build Agents ---
def create_agent(llm, tools, system_prompt):
    # This creates a standard ReAct agent (Reason -> Act)
    # It uses the system prompt to define the persona
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    return create_react_agent(llm, tools, prompt=prompt)

# --- 3. Create the Specialized Agents ---
research_agent = create_agent(
    llm,
    [web_search],
    "You are a web researcher. You search for accurate data. When you find data, present it clearly and concisely, indicating it is ready for further processing by another agent, such as plotting. Do not attempt to visualize or plot the data yourself."
)

coding_agent = create_agent(
    llm,
    [python_repl],
    "You are a Data Scientist. You write code to visualize and chart data."
)

# --- 4. Define Node Wrappers ---
# These functions bridge the Agent output to the Graph State
def research_node(state: AgentState):
    result = research_agent.invoke(state)
    # We return the last message (the agent's result) to the global state
    return {"messages": [result["messages"][-1]]}

def coding_node(state: AgentState):
    result = coding_agent.invoke(state)
    return {"messages": [result["messages"][-1]]}

print("✅ Workers (Researcher & Coder) Created.")

✅ Workers (Researcher & Coder) Created.


/tmp/ipython-input-783237345.py:32: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  return create_react_agent(llm, tools, prompt=prompt)


## Step 3: The Supervisor (The Router)

This is the brain of the operation.
The Supervisor is an LLM Chain that:
1.  Reads the conversation history.
2.  Decides which worker should act next.
3.  Or decides to `FINISH` if the user's request is satisfied.

We use **OpenAI Function Calling** to force the LLM to output a structured decision (`next`).

In [23]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Researcher", "Coder"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the user request, determine which worker"
    " should act next. Each worker will perform a task and respond with their results."
    " Based on the conversation history and the user's initial request, decide whether:"
    " 1. Another worker needs to perform a task (e.g., Researcher to gather data, Coder to plot)."
    " 2. The original request has been fully satisfied and the process should FINISH."
    " Prioritize completing the original request, like finding data and then plotting it."
    " If the Researcher has found data, and plotting is required, delegate to the Coder."
    " If all aspects of the request are handled, then FINISH."
)

# --- Define the Routing Schema ---
# This tells the LLM: "You MUST pick one of these options."
options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [{"enum": options}],
            }
        },
        "required": ["next"],
    },
}

# --- Build the Chain ---
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? Or should we FINISH? Select one: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind(functions=[function_def], function_call={"name": "route"})
    | JsonOutputFunctionsParser()
)

print("✅ Supervisor Chain Created.")

✅ Supervisor Chain Created.


## Step 4: The Graph Architecture

We wire the nodes in a **Star Topology**:
1.  **Supervisor** is the center.
2.  **Workers** (Researcher, Coder) are the spokes.
3.  After a Worker finishes, they **always** report back to the Supervisor.

In [24]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(AgentState)

# 1. Add Nodes
workflow.add_node("Supervisor", supervisor_chain)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", coding_node)

# 2. Add Edges
# Start at Supervisor
workflow.add_edge(START, "Supervisor")

# Workers always go back to Supervisor
workflow.add_edge("Researcher", "Supervisor")
workflow.add_edge("Coder", "Supervisor")

# 3. Conditional Logic (The Routing)
# Based on the 'next' field from the Supervisor, where do we go?
workflow.add_conditional_edges(
    "Supervisor",
    lambda x: x["next"],
    {
        "Researcher": "Researcher",
        "Coder": "Coder",
        "FINISH": END
    }
)

# 4. Compile
app = workflow.compile()

print("✅ Multi-Agent Graph Compiled.")

✅ Multi-Agent Graph Compiled.


## Step 5: Execution

Let's test the system with a multi-step request:
**"Research the stock price of Apple and then plot a chart."**

Watch the output carefully:
1.  Supervisor -> Researcher (to get data)
2.  Researcher -> Supervisor (returns data)
3.  Supervisor -> Coder (to plot data)
4.  Coder -> Supervisor (returns success)
5.  Supervisor -> FINISH

In [25]:
print("--- 🚀 Starting Multi-Agent Workflow ---")
initial_state = {"messages": [("user", "Research the stock price of Apple and then plot a chart.")]}

# We stream the output to see the steps
for s in app.stream(initial_state):
    if "__end__" not in s:
        # Print the node name and the output
        node_name = list(s.keys())[0]
        print(f"\n📍 Node: {node_name}")

        if node_name == "Supervisor":
            print(f"   Decision: {s[node_name]['next']}")
        else:
            # Print the last message from the worker
            last_msg = s[node_name]['messages'][0]
            # Handle both ToolMessages and AIMessages
            content = getattr(last_msg, 'content', str(last_msg))
            print(f"   Output: {content[:100]}...") # Truncate for readability

--- 🚀 Starting Multi-Agent Workflow ---

📍 Node: Supervisor
   Decision: Researcher
    🔎 [Tool] Searching for: Apple stock price

📍 Node: Researcher
   Output: Here is a simple line chart representing the recent stock prices of Apple:

```plaintext
Price ($)
 ...

📍 Node: Supervisor
   Decision: Researcher
    🔎 [Tool] Searching for: Apple stock price

📍 Node: Researcher
   Output: Here is a simple line chart representing the recent stock prices of Apple:

```plaintext
Price ($)
 ...

📍 Node: Supervisor
   Decision: Researcher
    🔎 [Tool] Searching for: Apple stock price

📍 Node: Researcher
   Output: Here is a simple line chart representing the recent stock prices of Apple:

```plaintext
Price ($)
 ...

📍 Node: Supervisor
   Decision: Researcher
    🔎 [Tool] Searching for: Apple stock price

📍 Node: Researcher
   Output: Here is a simple line chart representing the recent stock prices of Apple:

```plaintext
Price ($)
 ...

📍 Node: Supervisor
   Decision: Researcher
    🔎 [Tool] Sea

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT

## Summary

You have built a **Hierarchical Multi-Agent System**.

**Why is this better than a single agent?**
1.  **Separation of Concerns:** The Coder doesn't need to know how to Search. The Researcher doesn't need to know Python.
2.  **Context Management:** The Supervisor keeps the team focused.
3.  **Modularity:** You can easily add a "Writer" or "Reviewer" agent just by adding a node and updating the `members` list.

This architecture is the foundation for building complex, enterprise-grade AI assistants.